In [1]:
%run ColumnTransformers.ipynb
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score

In [2]:
def XgboostSearch(scale_pos_weight,X,y,model__early_stopping=15): 
    param_grid_xgb = {
        'model__n_estimators': [100, 300,500],  
        'model__learning_rate': [0.001,0.01],  
        'model__max_depth': [2,3,4], 
        'model__subsample': [0.7, 1.0],  
        'model__colsample_bytree': [0.6, 1.0],  
        'model__gamma': [0, 0.1, 0.5 ], 
        'model__reg_lambda': [0,1, 4], 
        'model__scale_pos_weight':[scale_pos_weight], # weight = 1 if negstive else wegiht=scale_pos_weight
        }  
    grid_search_XgBoost = GridSearchgGradient(XGBClassifier(eval_metric='auc',random_state=42,
                model__early_stopping_rounds=model__early_stopping),param_grid_xgb,X,y)
    return grid_search_XgBoost


def LightgbmSearch(ModelPipelineX,y):
    param_grid_lgb = {
        'model__n_estimators': [100, 300, 500],
        'model__learning_rate': [0.001, 0.01],
        'model__max_depth': [2, 3, 4],
        'model__subsample': [0.7, 1.0],
        'model__colsample_bytree': [0.6, 1.0],
        'model__min_split_gain': [0, 0.1, 0.5],   # similar to gamma from XGB
        'model__reg_lambda': [0, 1, 4], 
        'is_unbalance':[True] , 
        'model__early_stopping_rounds': [15]}

    grid_search_LightGBM = GridSearchgGradient(LGBMClassifier(eval_metric='auc',random_state=42),param_grid_lgb,X,y)
    return grid_search_LightGBM

def AdaSearch(ModelPipeline) : 
    param_grid_ada = {
    'model__n_estimators': [100,200], 
    'model__learning_rate': [0.6,1,2],  # before amount of say
    'model__estimator': [DecisionTreeClassifier(max_depth=2), 
                          DecisionTreeClassifier(max_depth=3),
                          DecisionTreeClassifier(max_depth=5)]
    }
    grid_search_ada = GridSearchCV(ModelPipeline, param_grid_ada, cv=5, scoring='roc_auc', n_jobs=3)
    return grid_search_ada

In [3]:
def kfold_split(X, y, n_splits=5, shuffle=True, random_state=42): 
    n_splits=5 
    n_samples = len(X)
    indices = np.arange(n_samples)
    
    if shuffle:
      rng = np.random.default_rng(seed=random_state) 
      rng.shuffle(indices)
    
    fold_sizes = np.full(n_splits, n_samples // n_splits, dtype=int)  # creating kinda equal split
    fold_sizes[:n_samples % n_splits] +=1  # distribute the remainder among the first folds
    current = 0
    fold_sizes=np.cumsum(fold_sizes)
    ResultMatrix=[]
    for fold_size in fold_sizes: 
        X_test=X.iloc[indices[current:fold_size]]
        y_test=y.iloc[indices[current:fold_size]]
        X_train=X.iloc[np.concatenate((indices[:current],indices[fold_size:]))]
        y_train=y.iloc[np.concatenate((indices[:current],indices[fold_size:]))]
        ResultMatrix.append((X_train, X_test, y_train, y_test))
        current=fold_size
    return ResultMatrix

In [4]:
def GridSearchgGradient(GradientModel,params,X,y, n_splits=5):
    BestModelParams=None 
    BestAucScore=float("-inf")
    for param in  list(ParameterGrid(params)): 
        Model=PipelineModel(GradientModel) 
        Model.set_params(**param)
        SplitMatrix=kfold_split(X,y, n_splits)
        aucVector=np.zeros(n_splits)
        counter=0
        for split in SplitMatrix:
            X_train, X_test, y_train, y_test=split[0],split[1],split[2],split[3]
            plct=PipeLineColumnTransformer() 
            plct.fit(X_test) 
            X_test=plct.transform(X_test)
            print("123")
            Model.fit(X_train,y_train,model__eval_set=[(X_test, y_test)]) 
            y_proba = Model.predict_proba(X_test)[:, 1]
            auc = roc_auc_score(y_test, y_proba)
            aucVector[counter]=auc 
            counter+=1
            
        if BestAucScore<np.mean(aucVector):
            BestAucScore=auc
            BestModel=param
    return BestAucScore,BestModel

In [5]:
X,y=KCrossData()
XgboostSearch(GetScalePosWeight(y),X,y)

123


XGBoostError: [17:55:12] C:\actions-runner\_work\xgboost\xgboost\src\data\quantile_dmatrix.cc:55: Check failed: ref->Info().num_col_ == n_features (30 vs. 39) : Invalid ref DMatrix, different number of features.